##### Load required packages

In [1]:
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from cmath import nan
from sklearn.metrics import confusion_matrix

##### Load data

In [2]:
'''
def load_data():
    mat_load = sio.loadmat('C:/Users/Zephyrus/Desktop/data/allTraces.mat')['allTraces']
    roi_load = sio.loadmat('C:/Users/Zephyrus/Desktop/data/allROIPositions.mat')['allROIPositions']
    ind_load = sio.loadmat('C:/Users/Zephyrus/Desktop/data/blFinalHit.mat')['blFinalHit']
    list1 = sio.loadmat('C:/Users/Zephyrus/Desktop/data/indicesSession1.mat')['indSess1']
    list2 = sio.loadmat('C:/Users/Zephyrus/Desktop/data/List2.mat')['List2']
    list3 = sio.loadmat('C:/Users/Zephyrus/Desktop/data/List3.mat')['List3']
    list4 = sio.loadmat('C:/Users/Zephyrus/Desktop/data/List4.mat')['List4']
    return mat_load, roi_load, ind_load, list1, list2, list3, list4
''' 

"\ndef load_data():\n    mat_load = sio.loadmat('C:/Users/Zephyrus/Desktop/data/allTraces.mat')['allTraces']\n    roi_load = sio.loadmat('C:/Users/Zephyrus/Desktop/data/allROIPositions.mat')['allROIPositions']\n    ind_load = sio.loadmat('C:/Users/Zephyrus/Desktop/data/blFinalHit.mat')['blFinalHit']\n    list1 = sio.loadmat('C:/Users/Zephyrus/Desktop/data/indicesSession1.mat')['indSess1']\n    list2 = sio.loadmat('C:/Users/Zephyrus/Desktop/data/List2.mat')['List2']\n    list3 = sio.loadmat('C:/Users/Zephyrus/Desktop/data/List3.mat')['List3']\n    list4 = sio.loadmat('C:/Users/Zephyrus/Desktop/data/List4.mat')['List4']\n    return mat_load, roi_load, ind_load, list1, list2, list3, list4\n"

In [2]:
def load_data():
    mat_load = sio.loadmat('C:/Users/m.nedeljkovic/Desktop/data/allTraces.mat')['allTraces']
    roi_load = sio.loadmat('C:/Users/m.nedeljkovic/Desktop/data/allROIPositions.mat')['allROIPositions']
    ind_load = sio.loadmat('C:/Users/m.nedeljkovic/Desktop/data/blFinalHit.mat')['blFinalHit']
    list1 = sio.loadmat('C:/Users/m.nedeljkovic/Desktop/data/indicesSession1.mat')['indSess1']
    list2 = sio.loadmat('C:/Users/m.nedeljkovic/Desktop/data/List2.mat')['List2']
    list3 = sio.loadmat('C:/Users/m.nedeljkovic/Desktop/data/List3.mat')['List3']
    list4 = sio.loadmat('C:/Users/m.nedeljkovic/Desktop/data/List4.mat')['List4']
    return mat_load, roi_load, ind_load, list1, list2, list3, list4

## Data preparation for the CNN models

- 1 region, all info about speakers, 4 sessions, white noise, 50 info per neuron (per speaker), 100 neurons per region => image 100x250(5x50) - 8 of them (2 states x 4 sessions) x 30 (sampling points over time)
- final 240 images (120 per state)
- oversampling input images with NaNs (maybe random oversampling)
- representing input images as different stimulus (100 x 50)
- combination of 2 different regions in one image / mice????
- including bandpass noise into data representation 
- representing input images as different stimulus (100 x 50)

### 1. One image - all regions, all speakers, neurons (oversampling with NaNs) - 100x60 (400)
- approx 100 neurons per region x 60 (5 speakers x 12 regions)
- 50 values x 4 sessions x 2 states -> 200 images per state

In [4]:
#lenghts of each trial
from cmath import isinf

def data_per_trial(tr, mat_load, roi_load, ind_load, list1, list2, list3, list4):
    lista = [[k[0] for k in list1], [k[0] for k in list2], [k[0] for k in list3], [k[0] for k in list4]]
    index = [0, 118, 88, 100, 98, 109, 78, 121, 76, 113, 97, 112, 118]

    for trial in range(1,len(index)):
        
        index[trial] = index[trial] + index[trial-1]
        dat_tr = mat_load[index[trial-1] : index[trial]+1, :, :, 0, :, :, :]
        ind_tr = ind_load[index[trial-1] : index[trial]+1, :, :, 0, :, :]

        #choose state
        dat_stan = dat_tr[:, 0, :, :, :, :]
        dat_staw = dat_tr[:, 1, :, :, :, :]
        ind_stan = ind_tr[:, 0, :, :, :]
        ind_staw = ind_tr[:, 1, :, :, :]

        br = 1

        if(trial == tr):
            for sesion in range(4):
                
                lista_ = lista[sesion]
                #choose session
                dat_sean = dat_stan[:, sesion, :, :, :]
                dat_seaw = dat_staw[:, sesion, :, :, :]
                ind_sean = ind_stan[:, sesion, :, :]
                ind_seaw = ind_staw[:, sesion, :, :]

                for sample in range(50):

                    dat_sean_df = dat_sean[:, :, sample, :]
                    dat_seaw_df = dat_seaw[:, :, sample, :]
                    ind_sean_df = ind_sean[:, :, sample]
                    ind_seaw_df = ind_seaw[:, :, sample]
                    ind_sean_df = ind_sean_df.astype('float')
                    ind_seaw_df = ind_seaw_df.astype('float')
                    ind_sean_df[ind_sean_df == 0] = 'nan' 
                    ind_seaw_df[ind_seaw_df == 0] = 'nan' 

                    dat_sean_df_ = dat_sean_df[0:1,:,:]
                    dat_seaw_df_ = dat_seaw_df[0:1,:,:]
                    ind_sean_df_ = ind_sean_df[0:1,:]
                    ind_seaw_df_ = ind_seaw_df[0:1,:]
                    for matrix in range(0,np.shape(dat_sean_df)[0]-1):
                        if(index[trial-1]+matrix in lista_):
                            dat_sean_df_ = np.append(dat_sean_df_, dat_sean[matrix][:, sample, :].reshape(1,dat_sean[matrix][:, sample, :].shape[0],dat_sean[matrix][:, sample, :].shape[1]), axis=0)
                            dat_seaw_df_ = np.append(dat_seaw_df_, dat_seaw[matrix][:, sample, :].reshape(1,dat_seaw[matrix][:, sample, :].shape[0],dat_seaw[matrix][:, sample, :].shape[1]), axis=0)
                            ind_sean_df_ = np.append(ind_sean_df_, ind_sean[matrix][:, sample].reshape(1, ind_sean[matrix][:, sample].shape[0]), axis=0)
                            ind_seaw_df_ = np.append(ind_seaw_df_, ind_seaw[matrix][:, sample].reshape(1, ind_seaw[matrix][:, sample].shape[0]), axis=0)
                    dat_sean_df = dat_sean_df_[1:,:,:]
                    dat_seaw_df = dat_seaw_df_[1:,:,:]
                    ind_sean_df = ind_sean_df_[1:,:]
                    ind_seaw_df = ind_seaw_df_[1:,:]
                
                    for time in range(31):
                        dat_sean_df[:,:,time] = np.multiply(np.array(dat_sean_df[:,:,time]),ind_sean_df)
                        dat_seaw_df[:,:,time] = np.multiply(np.array(dat_seaw_df[:,:,time]),ind_seaw_df)
                            
                    dat_sean_df = dat_sean_df.reshape(dat_sean_df.shape[0],155)
                    dat_seaw_df = dat_seaw_df.reshape(dat_seaw_df.shape[0],155)
                    dat_sean_df[dat_sean_df == np.inf] = 0
                    dat_seaw_df[dat_seaw_df == np.inf] = 0
                    if(np.any(dat_seaw_df == np.inf)==True or np.any(dat_sean_df == np.inf)==True or np.any(dat_sean_df == -np.inf)==True or np.any(dat_seaw_df == -np.inf)==True):
                        print(1)

                    file1 = "C:/Users/m.nedeljkovic/Desktop/cnn data/data/an" + str(br) + ".npy"
                    file2 = "C:/Users/m.nedeljkovic/Desktop/cnn data/data/aw" + str(br) + ".npy"
                    #file1 = "C:/Users/Zephyrus/Desktop/cnn data/data/an" + str(br) + ".npy"
                    #file2 = "C:/Users/Zephyrus/Desktop/cnn data/data/aw" + str(br) + ".npy"
                    
                    np.save(file1, dat_sean_df)
                    np.save(file2, dat_seaw_df)
                    br = br + 1
            

In [5]:
mat_load, roi_load, ind_load, list1, list2, list3, list4 = load_data()
data_per_trial(1, mat_load, roi_load, ind_load, list1, list2, list3, list4)

C:\Users\m.nedeljkovic\AppData\Local\Temp\ipykernel_19276\3636843975.py:60: RuntimeWarning: invalid value encountered in multiply
  dat_seaw_df[:,:,time] = np.multiply(np.array(dat_seaw_df[:,:,time]),ind_seaw_df)


### 2. One region -  training on one session, evaluation on different 

In [6]:
#lenghts of each trial
from cmath import isinf

def data_per_ses(tr, ses, mat_load, roi_load, ind_load, list1, list2, list3, list4):
    lista = [[k[0] for k in list1], [k[0] for k in list2], [k[0] for k in list3], [k[0] for k in list4]]
    index = [0, 118, 88, 100, 98, 109, 78, 121, 76, 113, 97, 112, 118]

    for trial in range(1,len(index)):
        
        index[trial] = index[trial] + index[trial-1]
        dat_tr = mat_load[index[trial-1] : index[trial]+1, :, :, 0, :, :, :]
        ind_tr = ind_load[index[trial-1] : index[trial]+1, :, :, 0, :, :]

        #choose state
        dat_stan = dat_tr[:, 0, :, :, :, :]
        dat_staw = dat_tr[:, 1, :, :, :, :]
        ind_stan = ind_tr[:, 0, :, :, :]
        ind_staw = ind_tr[:, 1, :, :, :]

        br = 1

        if(trial == tr):
            for sesion in range(4):
                
                if(sesion ==  ses):

                    lista_ = lista[sesion]
                    #choose session
                    dat_sean = dat_stan[:, sesion, :, :, :]
                    dat_seaw = dat_staw[:, sesion, :, :, :]
                    ind_sean = ind_stan[:, sesion, :, :]
                    ind_seaw = ind_staw[:, sesion, :, :]

                    for sample in range(50):

                        dat_sean_df = dat_sean[:, :, sample, :]
                        dat_seaw_df = dat_seaw[:, :, sample, :]
                        ind_sean_df = ind_sean[:, :, sample]
                        ind_seaw_df = ind_seaw[:, :, sample]
                        ind_sean_df = ind_sean_df.astype('float')
                        ind_seaw_df = ind_seaw_df.astype('float')
                        ind_sean_df[ind_sean_df == 0] = 'nan' 
                        ind_seaw_df[ind_seaw_df == 0] = 'nan' 

                        dat_sean_df_ = dat_sean_df[0:1,:,:]
                        dat_seaw_df_ = dat_seaw_df[0:1,:,:]
                        ind_sean_df_ = ind_sean_df[0:1,:]
                        ind_seaw_df_ = ind_seaw_df[0:1,:]
                        for matrix in range(0,np.shape(dat_sean_df)[0]-1):
                            if(index[trial-1]+matrix in lista_):
                                dat_sean_df_ = np.append(dat_sean_df_, dat_sean[matrix][:, sample, :].reshape(1,dat_sean[matrix][:, sample, :].shape[0],dat_sean[matrix][:, sample, :].shape[1]), axis=0)
                                dat_seaw_df_ = np.append(dat_seaw_df_, dat_seaw[matrix][:, sample, :].reshape(1,dat_seaw[matrix][:, sample, :].shape[0],dat_seaw[matrix][:, sample, :].shape[1]), axis=0)
                                ind_sean_df_ = np.append(ind_sean_df_, ind_sean[matrix][:, sample].reshape(1, ind_sean[matrix][:, sample].shape[0]), axis=0)
                                ind_seaw_df_ = np.append(ind_seaw_df_, ind_seaw[matrix][:, sample].reshape(1, ind_seaw[matrix][:, sample].shape[0]), axis=0)
                        dat_sean_df = dat_sean_df_[1:,:,:]
                        dat_seaw_df = dat_seaw_df_[1:,:,:]
                        ind_sean_df = ind_sean_df_[1:,:]
                        ind_seaw_df = ind_seaw_df_[1:,:]
                    
                        for time in range(31):
                            dat_sean_df[:,:,time] = np.multiply(np.array(dat_sean_df[:,:,time]),ind_sean_df)
                            dat_seaw_df[:,:,time] = np.multiply(np.array(dat_seaw_df[:,:,time]),ind_seaw_df)
                                
                        dat_sean_df = dat_sean_df.reshape(dat_sean_df.shape[0],155)
                        dat_seaw_df = dat_seaw_df.reshape(dat_seaw_df.shape[0],155)
                        dat_sean_df[dat_sean_df == np.inf] = 0
                        dat_seaw_df[dat_seaw_df == np.inf] = 0
                        if(np.any(dat_seaw_df == np.inf)==True or np.any(dat_sean_df == np.inf)==True or np.any(dat_sean_df == -np.inf)==True or np.any(dat_seaw_df == -np.inf)==True):
                            print(1)
                
                        file1 = "C:/Users/m.nedeljkovic/Desktop/cnn data/data/an" + str(br) + ".npy"
                        file2 = "C:/Users/m.nedeljkovic/Desktop/cnn data/data/aw" + str(br) + ".npy"
                        #file1 = "C:/Users/Zephyrus/Desktop/cnn data/data/an" + str(br) + ".npy"
                        #file2 = "C:/Users/Zephyrus/Desktop/cnn data/data/aw" + str(br) + ".npy"
                      
                        np.save(file1, dat_sean_df)
                        np.save(file2, dat_seaw_df)
                        br = br + 1
       


In [7]:
mat_load, roi_load, ind_load, list1, list2, list3, list4 = load_data()
data_per_ses(1, 0, mat_load, roi_load, ind_load, list1, list2, list3, list4)

C:\Users\m.nedeljkovic\AppData\Local\Temp\ipykernel_19276\943347217.py:62: RuntimeWarning: invalid value encountered in multiply
  dat_seaw_df[:,:,time] = np.multiply(np.array(dat_seaw_df[:,:,time]),ind_seaw_df)


### 3. One region - classification on different speakers

In [8]:
#lenghts of each trial
from cmath import isinf

def data_speak(tr, mat_load, roi_load, ind_load, list1, list2, list3, list4):
    lista = [[k[0] for k in list1], [k[0] for k in list2], [k[0] for k in list3], [k[0] for k in list4]]
    index = [0, 118, 88, 100, 98, 109, 78, 121, 76, 113, 97, 112, 118]

    for trial in range(1,len(index)):
        
        index[trial] = index[trial] + index[trial-1]
        dat_tr = mat_load[index[trial-1] : index[trial]+1, :, :, 0, :, :, :]
        ind_tr = ind_load[index[trial-1] : index[trial]+1, :, :, 0, :, :]

        #choose state
        dat_stan = dat_tr[:, 0, :, :, :, :]
        dat_staw = dat_tr[:, 1, :, :, :, :]
        ind_stan = ind_tr[:, 0, :, :, :]
        ind_staw = ind_tr[:, 1, :, :, :]

        br = 1

        if(trial == tr):
            for sesion in range(4):
                
                lista_ = lista[sesion]
                #choose session
                dat_sean = dat_stan[:, sesion, :, :, :]
                dat_seaw = dat_staw[:, sesion, :, :, :]
                ind_sean = ind_stan[:, sesion, :, :]
                ind_seaw = ind_staw[:, sesion, :, :]

                for sample in range(50):

                    dat_sean_df = dat_sean[:, :, sample, :]
                    dat_seaw_df = dat_seaw[:, :, sample, :]
                    ind_sean_df = ind_sean[:, :, sample]
                    ind_seaw_df = ind_seaw[:, :, sample]
                    ind_sean_df = ind_sean_df.astype('float')
                    ind_seaw_df = ind_seaw_df.astype('float')
                    ind_sean_df[ind_sean_df == 0] = 'nan' 
                    ind_seaw_df[ind_seaw_df == 0] = 'nan' 

                    dat_sean_df_ = dat_sean_df[0:1,:,:]
                    dat_seaw_df_ = dat_seaw_df[0:1,:,:]
                    ind_sean_df_ = ind_sean_df[0:1,:]
                    ind_seaw_df_ = ind_seaw_df[0:1,:]
                
                    for matrix in range(0,np.shape(dat_sean_df)[0]-1):
                        if(index[trial-1]+matrix in lista_):
                            dat_sean_df_ = np.append(dat_sean_df_, dat_sean[matrix][:, sample, :].reshape(1,dat_sean[matrix][:, sample, :].shape[0],dat_sean[matrix][:, sample, :].shape[1]), axis=0)
                            dat_seaw_df_ = np.append(dat_seaw_df_, dat_seaw[matrix][:, sample, :].reshape(1,dat_seaw[matrix][:, sample, :].shape[0],dat_seaw[matrix][:, sample, :].shape[1]), axis=0)
                            ind_sean_df_ = np.append(ind_sean_df_, ind_sean[matrix][:, sample].reshape(1, ind_sean[matrix][:, sample].shape[0]), axis=0)
                            ind_seaw_df_ = np.append(ind_seaw_df_, ind_seaw[matrix][:, sample].reshape(1, ind_seaw[matrix][:, sample].shape[0]), axis=0)

                    dat_sean_dfn = dat_sean_df_[1:,:,:]
                    dat_seaw_dfn = dat_seaw_df_[1:,:,:]
                    ind_sean_dfn = ind_sean_df_[1:,:]
                    ind_seaw_dfn = ind_seaw_df_[1:,:]

                    for time in range(31):
                        dat_sean_dfn[:,:,time] = np.multiply((dat_sean_dfn[:,:,time]),ind_sean_dfn)
                        dat_seaw_dfn[:,:,time] = np.multiply(np.array(dat_seaw_dfn[:,:,time]),ind_seaw_dfn)

                    for speak in range(0,5):

                        dat_sean_res = dat_sean_dfn[:,speak,:]
                        dat_seaw_res = dat_seaw_dfn[:,speak,:]
                        
                        dat_sean_res[dat_sean_res == np.inf] = 0
                        dat_seaw_res[dat_seaw_res == np.inf] = 0
                        if(np.any(dat_seaw_res == np.inf)==True or np.any(dat_sean_res == np.inf)==True or np.any(dat_sean_res == -np.inf)==True or np.any(dat_seaw_res == -np.inf)==True):
                            print(1)

                        file1 = "C:/Users/m.nedeljkovic/Desktop/cnn data/data/an" + str(br) + '_' + "sp" + str(speak) +  ".npy"
                        #file2 = "C:/Users/m.nedeljkovic/Desktop/cnn data/data/aw" + str(br) + '_' + "sp" + str(speak) +  ".npy"
                        #file1 = "C:/Users/Zephyrus/Desktop/cnn data/data/an" + str(br) + ".npy"
                        #file2 = "C:/Users/Zephyrus/Desktop/cnn data/data/aw" + str(br) + ".npy"
                    
                        np.save(file1, dat_sean_res)
                        #np.save(file2, dat_seaw_res)
                    
                    br = br + 1
        

In [9]:
mat_load, roi_load, ind_load, list1, list2, list3, list4 = load_data()
data_speak(1, mat_load, roi_load, ind_load, list1, list2, list3, list4)

C:\Users\m.nedeljkovic\AppData\Local\Temp\ipykernel_1944\682504556.py:62: RuntimeWarning: invalid value encountered in multiply
  dat_seaw_dfn[:,:,time] = np.multiply(np.array(dat_seaw_dfn[:,:,time]),ind_seaw_dfn)


### 3. All regions - classification on different speakers

In [13]:
#lenghts of each trial
from cmath import isinf

def data_speak_allreg(mat_load, roi_load, ind_load, list1, list2, list3, list4):
    lista = [[k[0] for k in list1], [k[0] for k in list2], [k[0] for k in list3], [k[0] for k in list4]]
    index = [0, 118, 88, 100, 98, 109, 78, 121, 76, 113, 97, 112, 118]

    br = 1

    for trial in range(1,len(index)):
        
        index[trial] = index[trial] + index[trial-1]
        dat_tr = mat_load[index[trial-1] : index[trial]+1, :, :, 0, :, :, :]
        ind_tr = ind_load[index[trial-1] : index[trial]+1, :, :, 0, :, :]

        #choose state
        dat_stan = dat_tr[:, 0, :, :, :, :]
        dat_staw = dat_tr[:, 1, :, :, :, :]
        ind_stan = ind_tr[:, 0, :, :, :]
        ind_staw = ind_tr[:, 1, :, :, :]

        for sesion in range(4):
            
            lista_ = lista[sesion]
            #choose session
            dat_sean = dat_stan[:, sesion, :, :, :]
            dat_seaw = dat_staw[:, sesion, :, :, :]
            ind_sean = ind_stan[:, sesion, :, :]
            ind_seaw = ind_staw[:, sesion, :, :]

            for sample in range(50):

                dat_sean_df = dat_sean[:, :, sample, :]
                dat_seaw_df = dat_seaw[:, :, sample, :]
                ind_sean_df = ind_sean[:, :, sample]
                ind_seaw_df = ind_seaw[:, :, sample]
                ind_sean_df = ind_sean_df.astype('float')
                ind_seaw_df = ind_seaw_df.astype('float')
                ind_sean_df[ind_sean_df == 0] = 'nan' 
                ind_seaw_df[ind_seaw_df == 0] = 'nan' 

                dat_sean_df_ = dat_sean_df[0:1,:,:]
                dat_seaw_df_ = dat_seaw_df[0:1,:,:]
                ind_sean_df_ = ind_sean_df[0:1,:]
                ind_seaw_df_ = ind_seaw_df[0:1,:]
            
                for matrix in range(0,np.shape(dat_sean_df)[0]-1):
                    if(index[trial-1]+matrix in lista_):
                        dat_sean_df_ = np.append(dat_sean_df_, dat_sean[matrix][:, sample, :].reshape(1,dat_sean[matrix][:, sample, :].shape[0],dat_sean[matrix][:, sample, :].shape[1]), axis=0)
                        dat_seaw_df_ = np.append(dat_seaw_df_, dat_seaw[matrix][:, sample, :].reshape(1,dat_seaw[matrix][:, sample, :].shape[0],dat_seaw[matrix][:, sample, :].shape[1]), axis=0)
                        ind_sean_df_ = np.append(ind_sean_df_, ind_sean[matrix][:, sample].reshape(1, ind_sean[matrix][:, sample].shape[0]), axis=0)
                        ind_seaw_df_ = np.append(ind_seaw_df_, ind_seaw[matrix][:, sample].reshape(1, ind_seaw[matrix][:, sample].shape[0]), axis=0)

                dat_sean_dfn = dat_sean_df_[1:,:,:]
                dat_seaw_dfn = dat_seaw_df_[1:,:,:]
                ind_sean_dfn = ind_sean_df_[1:,:]
                ind_seaw_dfn = ind_seaw_df_[1:,:]

                for time in range(31):
                    dat_sean_dfn[:,:,time] = np.multiply((dat_sean_dfn[:,:,time]),ind_sean_dfn)
                    dat_seaw_dfn[:,:,time] = np.multiply(np.array(dat_seaw_dfn[:,:,time]),ind_seaw_dfn)

                for speak in range(0,5):

                    dat_sean_res = dat_sean_dfn[:,speak,:]
                    dat_seaw_res = dat_seaw_dfn[:,speak,:]
                    
                    dat_sean_res[dat_sean_res == np.inf] = 0
                    dat_seaw_res[dat_seaw_res == np.inf] = 0
                    if(np.any(dat_seaw_res == np.inf)==True or np.any(dat_sean_res == np.inf)==True or np.any(dat_sean_res == -np.inf)==True or np.any(dat_seaw_res == -np.inf)==True):
                        print(1)

                    file1 = "C:/Users/m.nedeljkovic/Desktop/cnn data/data/an" + str(br) + '_' + "sp" + str(speak) +  ".npy"
                    #file2 = "C:/Users/m.nedeljkovic/Desktop/cnn data/data/aw" + str(br) + '_' + "sp" + str(speak) +  ".npy"
                    #file1 = "C:/Users/Zephyrus/Desktop/cnn data/data/an" + str(br) + ".npy"
                    #file2 = "C:/Users/Zephyrus/Desktop/cnn data/data/aw" + str(br) + ".npy"
                
                    np.save(file1, dat_sean_res)
                    #np.save(file2, dat_seaw_res)
                
                br = br + 1
        

In [14]:
mat_load, roi_load, ind_load, list1, list2, list3, list4 = load_data()
data_speak_allreg(mat_load, roi_load, ind_load, list1, list2, list3, list4)

C:\Users\m.nedeljkovic\AppData\Local\Temp\ipykernel_19540\3254080164.py:61: RuntimeWarning: invalid value encountered in multiply
  dat_seaw_dfn[:,:,time] = np.multiply(np.array(dat_seaw_dfn[:,:,time]),ind_seaw_dfn)
